The following code cell ensures that the underlying `.py` file gets automatically reloaded after you save any changes.  

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd    
from model import initialize_model, call_llm
from transformers import pipeline
import torch

/Users/ethanyuxin/Documents/World/nlp_group19/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

def preprocess(file_path):
    data = pd.read_json(path_or_buf=file_path, lines=True)
    return data

In [4]:
data = preprocess("./data/train.jsonl")

In [5]:
print(set([x for x in data["label"]]))

{'background', 'result', 'method'}


In [6]:
print(data.head())

     source  citeEnd                                        sectionName  \
0  explicit    175.0                                       Introduction   
1  explicit     36.0  Novel Quantitative Trait Loci for Seminal Root...   
2  explicit    228.0                                       Introduction   
3  explicit    110.0                                         Discussion   
4  explicit    239.0                                         Discussion   

   citeStart                                             string       label  \
0      168.0  However, how frataxin interacts with the Fe-S ...  background   
1       16.0  In the study by Hickey et al. (2012), spikes w...  background   
2      225.0  The drug also reduces catecholamine secretion,...  background   
3       46.0  By clustering with lowly aggressive close kin ...  background   
4      234.0  Ophthalmic symptoms are rare manifestations of...  background   

   label_confidence                             citingPaperId  \
0        

In [7]:
print(data.iloc[0].string)

However, how frataxin interacts with the Fe-S cluster biosynthesis components remains unclear as direct one-to-one interactions with each component were reported (IscS [12,22], IscU/Isu1 [6,11,16] or ISD11/Isd11 [14,15]).


In [8]:
print(len(data))

8243


In [9]:
labels = []
reasonings = []
raw_output = []
client, _ = initialize_model("")
start = 50
end = start + 50
for i in range(start, end):
    current_data = data.iloc[i]
    classification, reasoning = call_llm(client, "", current_data.string, current_data.sectionName)
    raw_output.append(classification)
    labels.append(classification)
    reasonings.append(reasoning)

df = pd.DataFrame(zip(labels, reasonings), columns=["id", "reasoning"])
df.to_csv(f"{start}_{end}.csv")

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='{"classification": "background", "reasoning": "The text is providing a summary of the main distribution of 13-galactosidase-producing cells in tissues from different lines, which is typical of the background section in a scientific paper, where authors usually introduce the context and provide necessary context for the study."}', tool_calls=None), logprobs=None)], created=1743232015, id='', model='meta-llama/Meta-Llama-3-8B-Instruct', system_fingerprint='3.2.1-native', usage=ChatCompletionOutputUsage(completion_tokens=66, prompt_tokens=183, total_tokens=249), object='chat.completion')
ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='{"classification": "background", "reasoning": "The text discusses previous research and provides conte

In [10]:
print(labels)
print(reasonings)

['background', 'background', 'method', 'method', 'background', 'method', 'background', 'method', 'background', 'background', 'background', 'background', 'background', 'background', 'method', 'background', 'background', 'background', 'background', 'background', 'background', 'background', 'method', 'background', 'background', 'result', 'method', 'method', 'background', 'background', 'result', 'method', 'background', 'result', 'result', 'background', 'background', 'method', 'method', 'background', 'background', 'background', 'method', 'background', 'background', 'method', 'background', 'method', 'background', 'background']
['The text is providing a summary of the main distribution of 13-galactosidase-producing cells in tissues from different lines, which is typical of the background section in a scientific paper, where authors usually introduce the context and provide necessary context for the study.', 'The text discusses previous research and provides context for the significance of the

In [11]:
# reasonings = [" ".join([x.strip() for x in y[1:]]) for y in raw_output]

In [12]:
print(labels)

['background', 'background', 'method', 'method', 'background', 'method', 'background', 'method', 'background', 'background', 'background', 'background', 'background', 'background', 'method', 'background', 'background', 'background', 'background', 'background', 'background', 'background', 'method', 'background', 'background', 'result', 'method', 'method', 'background', 'background', 'result', 'method', 'background', 'result', 'result', 'background', 'background', 'method', 'method', 'background', 'background', 'background', 'method', 'background', 'background', 'method', 'background', 'method', 'background', 'background']


In [13]:
df = pd.DataFrame(zip(labels, reasonings), columns=["id", "reasoning"])
df.to_csv("first_50.csv", )